## 利用信息抽取技术搭建知识库

在这个notebook文件中，有些模板代码已经提供给你，但你还需要实现更多的功能来完成这个项目。除非有明确要求，你无须修改任何已给出的代码。以**'【练习】'**开始的标题表示接下来的代码部分中有你需要实现的功能。这些部分都配有详细的指导，需要实现的部分也会在注释中以'TODO'标出。请仔细阅读所有的提示。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

---

### 让我们开始吧

本项目的目的是结合命名实体识别、依存语法分析、实体消歧、实体统一对网站开放语料抓取的数据建立小型知识图谱。

在现实世界中，你需要拼凑一系列的模型来完成不同的任务；举个例子，用来预测狗种类的算法会与预测人类的算法不同。在做项目的过程中，你可能会遇到不少失败的预测，因为并不存在完美的算法和模型。你最终提交的不完美的解决方案也一定会给你带来一个有趣的学习经验！


---


## 步骤 1：实体统一

实体统一做的是对同一实体具有多个名称的情况进行统一，将多种称谓统一到一个实体上，并体现在实体的属性中（可以给实体建立“别称”属性）

例如：对“河北银行股份有限公司”、“河北银行公司”和“河北银行”我们都可以认为是一个实体，我们就可以将通过提取前两个称谓的主要内容，得到“河北银行”这个实体关键信息。

公司名称有其特点，例如后缀可以省略、上市公司的地名可以省略等等。在data/dict目录中提供了几个词典，可供实体统一使用。
- company_suffix.txt是公司的通用后缀词典
- company_business_scope.txt是公司经营范围常用词典
- co_Province_Dim.txt是省份词典
- co_City_Dim.txt是城市词典
- stopwords.txt是可供参考的停用词

### 练习1：
编写main_extract函数，实现对实体的名称提取“主体名称”的功能。

In [9]:
import jieba
import jieba.posseg as pseg
import re
import datetime

# 替换特殊符号
def remove_word(seg,stop_word,d_4_delete):
    temp_seg_lst = [word for word in seg if word not in stop_word]
    seg_lst = [word for word in temp_seg_lst if word not in d_4_delete]
    return seg_lst

# 实现公司名称中地名提前
def city_prov_ahead(seg,d_city_province):
    city_prov_lst = [word for word in seg if word in d_city_province]
    seg_lst = [word for word in seg if word not in d_city_province]
    #print(seg_lst)
    #print(city_prov_lst)
    return city_prov_lst+seg_lst


# 从输入的“公司名”中提取主体
def main_extract(input_str,stop_word,d_4_delete,d_city_province):
    # 开始分词并处理
    seg = jieba.lcut(input_str)
    seg_lst = remove_word(seg,stop_word,d_4_delete)
    seg_lst = city_prov_ahead(seg_lst,d_city_province)
    #print(seg_lst)
    return seg_lst

# 初始化，加载词典
def my_initial():
    fr1 = open(r"../data/dict/co_City_Dim.txt", encoding='utf-8')
    fr2 = open(r"../data/dict/co_Province_Dim.txt", encoding='utf-8')
    fr3 = open(r"../data/dict/company_business_scope.txt", encoding='utf-8')
    fr4 = open(r"../data/dict/company_suffix.txt", encoding='utf-8')
    #城市名
    lines1 = fr1.readlines()
    d_4_delete = []
    d_city_province = [re.sub(r'(\r|\n)*','',line) for line in lines1]
    #省份名
    lines2 = fr2.readlines()
    l2_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines2]
    d_city_province.extend(l2_tmp)
    #公司后缀
    lines3 = fr3.readlines()
    l3_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines3]
    lines4 = fr4.readlines()
    l4_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines4]
    d_4_delete.extend(l4_tmp)
    #get stop_word
    fr = open(r'../data/dict/stopwords.txt', encoding='utf-8')   
    stop_word = fr.readlines()
    stop_word_after = [re.sub(r'(\r|\n)*','',stop_word[i]) for i in range(len(stop_word))]
    stop_word_after[-1] = stop_word[-1]
    stop_word = stop_word_after
    return d_4_delete,stop_word,d_city_province


In [10]:
# TODO：测试实体统一用例
d_4_delete,stop_word,d_city_province = my_initial()
company_name = "协鑫集成北京科技股份有限公司"
lst = main_extract(company_name,stop_word,d_4_delete,d_city_province)
company_name = ''.join(lst)  # 对公司名提取主体部分，将包含相同主体部分的公司统一为一个实体
print(company_name)

Building prefix dict from the default dictionary ...
I0808 22:00:37.314595 140158294460224 __init__.py:111] Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
I0808 22:00:37.758519 140158294460224 __init__.py:145] Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.490 seconds.
I0808 22:00:37.806966 140158294460224 __init__.py:163] Loading model cost 0.490 seconds.
Prefix dict has been built succesfully.
I0808 22:00:37.807910 140158294460224 __init__.py:164] Prefix dict has been built succesfully.


北京协鑫集成科技股份


## 步骤 2：实体识别
有很多开源工具可以帮助我们对实体进行识别。常见的有LTP、StanfordNLP、FoolNLTK等等。

本次采用FoolNLTK实现实体识别，fool是一个基于bi-lstm+CRF算法开发的深度学习开源NLP工具，包括了分词、实体识别等功能，大家可以通过fool很好地体会深度学习在该任务上的优缺点。

在‘data/train_data.csv’和‘data/test_data.csv’中是从网络上爬虫得到的上市公司公告，数据样例如下：

In [3]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('../data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data.head()

,id,sentence,tag,member1,member2
0,6461,与本公司关系:受同一公司控制 2，杭州富生电器有限公司企业类型: 有限公司注册地址: 富阳市...,0,0,0
1,2111,三、关联交易标的基本情况 1、交易标的基本情况 公司名称:红豆集团财务有限公司 公司地址:无...,0,0,0
2,9603,2016年协鑫集成科技股份有限公司向瑞峰（张家港）光伏科技有限公司支付设备款人民币4，515...,1,协鑫集成科技股份有限公司,瑞峰（张家港）光伏科技有限公司
3,3456,证券代码:600777 证券简称:新潮实业 公告编号:2015-091 烟台新潮实业股份有限...,0,0,0
4,8844,本集团及广发证券股份有限公司持有辽宁成大股份有限公司股票的本期变动系买卖一揽子沪深300指数...,1,广发证券股份有限公司,辽宁成大股份有限公司


In [4]:
test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data.head()

,id,sentence
0,9259,2015年1月26日，多氟多化工股份有限公司与李云峰先生签署了《附条件生效的股份认购合同》
1,9136,2、2016年2月5日，深圳市新纶科技股份有限公司与侯毅先
2,220,2015年10月26日，山东华鹏玻璃股份有限公司与张德华先生签署了附条件生效条件的《股份认购合同》
3,9041,2、2015年12月31日，印纪娱乐传媒股份有限公司与肖文革签订了《印纪娱乐传媒股份有限公司...
4,10041,一、金发科技拟与熊海涛女士签订《股份转让协议》，协议约定：以每股1.0509元的收购价格，收...


我们选取一部分样本进行标注，即train_data，该数据由5列组成。id列表示原始样本序号；sentence列为我们截取的一段关键信息；如果关键信息中存在两个实体之间有股权交易关系则tag列为1，否则为0；如果tag为1，则在member1和member2列会记录两个实体出现在sentence中的名称。

剩下的样本没有标注，即test_data，该数据只有id和sentence两列，希望你能训练模型对test_data中的实体进行识别，并判断实体对之间有没有股权交易关系。

### 练习2：
将每句句子中实体识别出，存入实体词典，并用特殊符号替换语句。


In [11]:
import fool
test_lsting = "我上周五購買了一支基金，希望能掙到和女朋友一起去江西武功山玩的錢。"
words,ners = fool.analysis(test_lsting)
print(words)
print(ners)


[[('我', 'r'), ('上周', 't'), ('五購', 'w'), ('買', 'v'), ('了', 'y'), ('一', 'm'), ('支', 'q'), ('基金', 'n'), ('，', 'wd'), ('希望', 'v'), ('能', 'v'), ('掙', 'v'), ('到', 'v'), ('和', 'c'), ('女朋友', 'n'), ('一起', 'd'), ('去', 'v'), ('江西', 'ns'), ('武功山', 'ns'), ('玩', 'v'), ('的', 'ude'), ('錢', 'n'), ('。', 'wj')]]
[[(1, 5, 'time', '上周五'), (24, 27, 'location', '江西')]]


In [6]:
ners[0].sort(key=lambda x:x[0], reverse=False)

In [7]:
ners

[[(1, 5, 'time', '上周五'), (24, 27, 'location', '江西')]]

In [12]:
# 处理test数据，利用开源工具进行实体识别和并使用实体统一函数存储实体

import fool
import pandas as pd
from copy import copy


test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data['ner'] = None
ner_id = 1001
ner_dict_new = {}  # 存储所有实体統一後結果與其編號對應關係
ner_dict_reverse_new = {}  # 存储所有实体統一後結果與其未統一前实体名對應關係

for i in range(len(test_data)):
    sentence = copy(test_data.iloc[i, 1])
    sentence = sentence.strip()
    sentence = sentence.replace(' ','')#str.replace(old, new[, max])
    # 调用fool进行实体识别，得到words和ners结果
    words,ners = fool.analysis(sentence)
    ners[0].sort(key=lambda x:x[0], reverse=True) #True 從大到小
    for start, end, ner_type, ner_name in ners[0]:#(24, 27, 'location', '江西')
        if ner_type=='company' or ner_type=='person':
            # 调用实体统一函数，存储统一后的实体并自增ner_id
            ner_lst = main_extract(ner_name,stop_word,d_4_delete,d_city_province)
            company_main_name = ''.join(ner_lst)
            ## 存储所有实体統一後結果與其未統一前实体名對應關係
            if company_main_name not in ner_dict_reverse_new:
                ner_dict_reverse_new[company_main_name] = [ner_name]
            else :
                if ner_name not in ner_dict_reverse_new[company_main_name]:
                    ner_dict_reverse_new[company_main_name].append(ner_name)
            ## 存储所有实体統一後結果與其編號對應關係
            if company_main_name not in ner_dict_new:
                ner_dict_new[company_main_name] = ner_id
                ner_id +=1
            
            # 在句子中用编号替换实体名
            sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end-1:]
    test_data.iloc[i, -1] = sentence

X_test_sentence = test_data[['ner']]
X_test_sentence.head()


,ner
0,2015年1月26日， ner_1002_ 与 ner_1001_ 先生签署了《附条件生效的...
1,2、2016年2月5日， ner_1004_ 与 ner_1003_ 先
2,2015年10月26日， ner_1006_ 与 ner_1005_ 先生签署了附条件生效条...
3,2、2015年12月31日， ner_1008_ 与 ner_1007_ 签订了《印纪娱乐传...
4,一、 ner_1010_ 拟与 ner_1009_ 女士签订《股份转让协议》，协议约定：以每...


In [13]:
# 处理train数据，利用开源工具进行实体识别和并使用实体统一函数存储实体
train_data = pd.read_csv('../data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data['ner'] = None
for i in range(len(train_data)):
    # 判断正负样本
    if train_data.iloc[i,:]['member1']=='0' and train_data.iloc[i,:]['member2']=='0':
        sentence = copy(train_data.iloc[i, 1])
        sentence = sentence.strip()
        sentence = sentence.replace(' ','')#str.replace(old, new[, max])
        
        
        # TODO：调用fool进行实体识别，得到words和ners结果
        words,ners = fool.analysis(sentence)
        ners[0].sort(key=lambda x:x[0], reverse=True)
        for start, end, ner_type, ner_name in ners[0]:
            if ner_type=='company' or ner_type=='person':
                # TODO：调用实体统一函数，存储统一后的实体.并自增ner_id
                ner_lst = main_extract(ner_name,stop_word,d_4_delete,d_city_province)
                company_main_name = ''.join(ner_lst)
                ## 存储所有实体統一後結果與其未統一前实体名對應關係
                if company_main_name not in ner_dict_reverse_new:
                    ner_dict_reverse_new[company_main_name] = [ner_name]
                else :
                    if ner_name not in ner_dict_reverse_new[company_main_name]:
                        ner_dict_reverse_new[company_main_name].append(ner_name)
                ## 存储所有实体統一後結果與其編號對應關係
                if company_main_name not in ner_dict_new:
                    ner_dict_new[company_main_name] = ner_id
                    ner_id +=1

                # 在句子中用编号替换实体名
                sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end-1:]
            train_data.iloc[i,-1] = sentence   
    else:
        sentence = copy(train_data.iloc[i,:]['sentence'])
        sentence = sentence.strip()
        sentence = sentence.replace(' ','')#str.replace(old, new[, max])
        for ner_name in [train_data.iloc[i,:]['member1'],train_data.iloc[i,:]['member2']]:
            # TODO：调用实体统一函数，存储统一后的实体.并自增ner_id
            ner_lst = main_extract(ner_name,stop_word,d_4_delete,d_city_province)
            company_main_name = ''.join(ner_lst)
            ## 存储所有实体統一後結果與其未統一前实体名對應關係
            if company_main_name not in ner_dict_reverse_new:
                ner_dict_reverse_new[company_main_name] = [ner_name]
            else :
                if ner_name not in ner_dict_reverse_new[company_main_name]:
                    ner_dict_reverse_new[company_main_name].append(ner_name)
            ## 存储所有实体統一後結果與其編號對應關係
            if company_main_name not in ner_dict_new:
                ner_dict_new[company_main_name] = ner_id
                ner_id +=1

            # 在句子中用编号替换实体名 
            sentence = re.sub(company_main_name,' ner_%s_ '%(str(ner_dict_new[company_main_name])),sentence)
        train_data.iloc[i,-1] = sentence                

y_train = train_data.loc[:,['tag']]
train_num = len(train_data)
X_train_sentence = train_data[['ner']]

# 将train和test放在一起提取特征
X = pd.concat([X_train_sentence, X_test_sentence])


In [15]:
X.head()

,ner
0,与本公司关系:受同一公司控制2， ner_1611_ 企业类型:有限公司注册地址:富阳市东洲...
1,三、关联交易标的基本情况1、交易标的基本情况公司名称: ner_1613_ 地址:无锡市锡山...
2,2016年 ner_1614_ 有限公司向瑞峰（张家港）光伏科技有限公司支付设备款人民币4，...
3,证券代码:600777证券简称: ner_1198_ 公告编号:2015-091 ner_1...
4,本集团及 ner_1089_ 股份有限公司持有 ner_1616_ 大股份有限公司股票的本期...


## 步骤 3：关系抽取


目标：借助句法分析工具，和实体识别的结果，以及文本特征，基于训练数据抽取关系，并存储进图数据库。

本次要求抽取股权交易关系，关系为有向边，由投资方指向被投资方。

模板建立可以使用“正则表达式”、“实体间距离”、“实体上下文”、“依存句法”等。

答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
- info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
- info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。

示例：
- info_extract_entity.csv

| 实体编号 | 实体名 |
| ------ | ------ |
| 1001 | 小王 |
| 1002 | A化工厂 |

- info_extract_submit.csv

| 实体1 | 实体2 |
| ------ | ------ |
| 1001 | 1003 |
| 1002 | 1001 |

### 练习3：提取文本tf-idf特征

去除停用词，并转换成tfidf向量。

In [19]:
# code
from sklearn.feature_extraction.text import TfidfVectorizer  
from pyltp import Segmentor
from pyltp import Postagger

# 实体符号加入分词词典
with open('../data/user_dict.txt', 'w') as fw:
    for v in ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']:
        fw.write( v + '号企业 ni\n')


postagger = Postagger() # 初始化实例
postagger.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/pos.model', '../data/user_dict.txt')  # 加载模型
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt')  # 加载模型
# 加载停用词
fr = open(r'../data/dict/stopwords.txt','r', encoding='utf-8')   
stop_word = fr.readlines()
stop_word = [re.sub(r'(\r|\n)*','',word) for word in stop_word]

# 分词
# f = lambda x: ' '.join([for word in segmentor.segment(x) if word not in stop_word])
# corpus=sample_data['sentence'].map(f).tolist()
def filterd_segment(s):
    #
    tmp_ner_dict = {}
    num_lst = ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']
    # 将公司代码替换为特殊称谓，保证分词词性正确
 
    for i, ner in enumerate(list(set(re.findall(r'(ner_\d\d\d\d\_)', s)))):
        try:
            tmp_ner_dict[num_lst[i]+'号企业'] = ner
        except IndexError:
            # 定义错误情况的输出
            print('error')
        s = s.replace(ner,num_lst[i]+'号企业')
    words = segmentor.segment(s)
    result_list = [tmp_ner_dict.get(word) if tmp_ner_dict.get(word) else word for word in words if word not in stop_word]
    resurt_segment = ' '.join(result_list)
    #segmentor.release()  # 释放模型
    return resurt_segment


corpus_train = X_train_sentence['ner'].map(filterd_segment).tolist()
corpus_test = X_test_sentence['ner'].map(filterd_segment).tolist()

# # 提取tfidf特征
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)

# print(X_test_sentence['ner'].map(filterd_segment).tolist())
#print(corpus_test)
print(X_train.shape)
print(X_test.shape)




(850, 4562)
(419, 4562)


In [20]:
 for i, ner in enumerate(list(set(re.findall(r'(ner_\d\d\d\d\_)',  X_train_sentence['ner'][1])))):
        print(i)
        print(ner)

0
ner_1612_
1
ner_1613_


In [18]:
# -*- coding: utf-8 -*-
from pyltp import Parser
from pyltp import Segmentor
from pyltp import Postagger
from itertools import combinations
from scipy import sparse
from tqdm import tqdm
import networkx as nx
import pylab
import re

#.......看看句法分析是個啥...........
postagger = Postagger() # 初始化实例
postagger.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/pos.model', '../data/user_dict.txt')  # 加载模型
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt')  # 加载模型

words = segmentor.segment(X_train_sentence['ner'][1])
tags = postagger.postag(words)
parser = Parser() # 初始化实例
parser.load('/home/zing/Downloads/ltp_data_v3.4.0/parser.model')  # 加载模型
arcs = parser.parse(words, tags)  # 句法分析
arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))

# 句法分析结果输出
parse_result = pd.DataFrame([[a,b,c,d] for a,b,c,d in zip(list(words),list(tags), arcs_lst[0], arcs_lst[1])],
                            index = range(1,len(words)+1))
parser.release()  # 释放模型
print(parse_result)
#print(arcs)
#print(arcs_lst)

[[5, 3, 5, 5, 7, 7, 15, 15, 8, 11, 13, 13, 14, 15, 0, 15, 29, 17, 20, 29, 20, 23, 24, 25, 29, 27, 28, 29, 15, 29, 32, 33, 35, 35, 49, 35, 49, 37, 49, 39, 42, 49, 42, 45, 48, 48, 48, 49, 29, 52, 52, 49, 49, 49, 54, 57, 55, 54, 60, 61, 54, 61, 66, 65, 63, 61, 66, 70, 70, 67, 73, 73, 67, 77, 77, 75, 67, 79, 80, 67, 66, 89, 84, 82, 82, 87, 85, 82, 66, 49, 96, 91, 92, 96, 96, 101, 96, 101, 100, 98, 49, 101, 101, 101, 106, 107, 109, 107, 104, 109, 112, 109, 112, 101, 118, 117, 115, 101, 120, 118, 122, 120, 101, 101, 126, 127, 129, 127, 130, 124, 130, 135, 135, 133, 131, 137, 138, 139, 131, 101, 101, 143, 145, 143, 141, 101, 150, 149, 147, 101, 150, 153, 151, 153, 150, 150, 158, 156, 156, 161, 162, 165, 162, 165, 150, 49, 172, 169, 172, 169, 172, 49, 49, 175, 49, 180, 179, 179, 180, 175, 180, 180, 49, 185, 186, 189, 186, 189, 49, 15], ['ATT', 'WP', 'ATT', 'ATT', 'ATT', 'ATT', 'ATT', 'ATT', 'WP', 'ATT', 'ATT', 'ATT', 'ATT', 'ATT', 'HED', 'WP', 'ATT', 'WP', 'ATT', 'ATT', 'WP', 'ATT', 'ATT', 'AT

### 练习4：提取句法特征
除了词语层面的句向量特征，我们还可以从句法入手，提取一些句法分析的特征。

参考特征：

1、企业实体间距离

2、企业实体间句法距离

3、企业实体分别和关键触发词的距离

4、实体的依存关系类别

In [21]:
# -*- coding: utf-8 -*-
from pyltp import Parser
from pyltp import Segmentor
from pyltp import Postagger
from itertools import combinations
from scipy import sparse
from tqdm import tqdm
import networkx as nx
import pylab
import re

postagger = Postagger() # 初始化实例
postagger.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/pos.model', '../data/user_dict.txt')  # 加载模型
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon('/home/zing/Downloads/ltp_data_v3.4.0/cws.model', '../data/user_dict.txt')  # 加载模型


def shortest_path(arcs_ret, source, target):
    """
    求出两个词最短依存句法路径，不存在路径返回-1
    arcs_ret：句法分析结果表格
    source：实体1
    target：实体2
    """
    G=nx.DiGraph()
    # 为这个网络添加节点...
    for i in list(arcs_ret.index):
        G.add_node(i)
        # 在网络中添加带权中的边...（注意，我们需要的是无向边）        
        G.add_edge(arcs_ret.loc[i, 2], i)
    
    G.to_undirected()  # 转换成无向图

    try:
        # 利用nx包中shortest_path_length方法实现最短距离提取
        
        distance = nx.shortest_path_length(G, source=source, target=target)
        return distance
    except:
        return -1


def parse(s):
    """
    对语句进行句法分析，并返回句法结果
    """
    tmp_ner_dict = {}
    num_lst = ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']

    # 将公司代码替换为特殊称谓，保证分词词性正确
    for i, ner in enumerate(list(set(re.findall(r'(ner_\d\d\d\d_)', s)))):
        try:
            tmp_ner_dict[num_lst[i]+'号企业'] = ner
        except IndexError:
            #定义错误情况的输出
            
            print('替换出错!')
            
        s = s.replace(ner, num_lst[i]+'号企业')
    words = segmentor.segment(s)
    tags = postagger.postag(words)
    parser = Parser() # 初始化实例
    parser.load('/home/zing/Downloads/ltp_data_v3.4.0/parser.model')  # 加载模型
    arcs = parser.parse(words, tags)  # 句法分析
    arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))
    
    # 句法分析结果输出
    parse_result = pd.DataFrame([[a,b,c,d] for a,b,c,d in zip(list(words), list(tags), arcs_lst[0], arcs_lst[1])], 
                                index = range(1,len(words)+1))
    parser.release()  # 释放模型
    
#     print(parse_result)
    # 提取企业实体依存句法类型


    # 投资关系关键词
    key_words = ["收购","竞拍","转让","扩张","并购","注资","整合","并入","竞购","竞买","支付","收购价","收购价格","承购","购得","购进", 
                 "购入","买进","买入","赎买","购销","议购","函购","函售","抛售","售卖","销售","转售"]
    # *提取关键词和对应句法关系提取特征
    # 
    # 初始化企业实体的索引列表
    ner_index_list = []
    # 初始化关键词的索引列表
    keyword_index_list = []
    
    # 遍历句法分析表格的索引
    for idx in parse_result.index:
        if parse_result.loc[idx, 0].endswith('号企业'):
            ner_index_list.append(idx)
        
        if parse_result.loc[idx, 0] in key_words:
            keyword_index_list.append(idx)
    
    # 1)在句子中的关键词数量
    parse_feature1 = len(keyword_index_list)
    
    # 2)若关键词存在
    # # 初始化判断与关键词有直接关系的'X号企业'句法类型为'S..'的数量
    parse_feature2 = 0
    # # 初始化判断与关键词有直接关系的'X号企业'句法类型为'.OB'的数量
    parse_feature3 = 0
    
    # 遍历出现在句子中的关键词索引
    for i in keyword_index_list:
        # 遍历出现在句子中的实体索引
        for j in ner_index_list:
            # 如果实体句法类型以S开头(主语)或者OB结尾(宾语)
            if parse_result.loc[j, 3].startswith('S') or parse_result.loc[j, 3].endswith('OB'):
                # 若关键词对应句法关联索引为实体索引，则parse_feature2数量+1
                if parse_result.loc[i, 2]==j:
                    parse_feature2 += 1
                # 若实体索引对应句法关联索引为关键词索引，则parse_feature3数量+1
                if parse_result.loc[j, 2]==i:
                    parse_feature3 += 1

    # 3)实体与关键词之间距离的平均值，最大值和最小值
    ner_keyword_pair_list = [(ner_index, keyword_index) for ner_index in ner_index_list for keyword_index in keyword_index_list]
    ner_keyword_distance_list = [abs(pair[0]-pair[1]) for pair in ner_keyword_pair_list]

    parse_feature4 = np.mean(ner_keyword_distance_list) if ner_keyword_distance_list else 0
    parse_feature5 = max(ner_keyword_distance_list) if ner_keyword_distance_list else 0
    parse_feature6 = min(ner_keyword_distance_list) if ner_keyword_distance_list else 0

    # 4)实体与关键词之间句法距离的平均值，最大值和最小值
    ner_keyword_parse_distance_list = [shortest_path(parse_result, pair[0], pair[1]) for pair in ner_keyword_pair_list]

    parse_feature7 = np.mean(ner_keyword_parse_distance_list) if ner_keyword_parse_distance_list else 0
    parse_feature8 = max(ner_keyword_parse_distance_list) if ner_keyword_parse_distance_list else 0
    parse_feature9 = min(ner_keyword_parse_distance_list) if ner_keyword_parse_distance_list else 0
    
    # 5)实体之间距离的平均值，最大值和最小值
    ner_pair_list = list(combinations(ner_index_list, 2))
    ner_distance_list = [abs(pair[0]-pair[1]) for pair in ner_pair_list]

    parse_feature10 = np.mean(ner_distance_list) if ner_distance_list else 0
    parse_feature11 = max(ner_distance_list) if ner_distance_list else 0
    parse_feature12 = min(ner_distance_list) if ner_distance_list else 0

    # 6)实体之间句法距离的平均值，最大值和最小值
    ner_parse_distance_list = [shortest_path(parse_result, pair[0], pair[1]) for pair in ner_pair_list]

    parse_feature13 = np.mean(ner_parse_distance_list) if ner_parse_distance_list else 0
    parse_feature14 = max(ner_parse_distance_list) if ner_parse_distance_list else 0
    parse_feature15 = min(ner_parse_distance_list) if ner_parse_distance_list else 0
    
    return [parse_feature1, parse_feature2, parse_feature3, 
            parse_feature4, parse_feature5, parse_feature6,
            parse_feature7, parse_feature8, parse_feature9,
            parse_feature10, parse_feature11, parse_feature12,
            parse_feature13, parse_feature14, parse_feature15]
    

#         0   1   2    3
# 1   2015年  nt   3  ATT
# 2      1月  nt   3  ATT
# 3     26日  nt   9  ADV
# 4       ，  wp   3   WP
# 5    二号企业  ni   9  SBV
# 6       与   p   9  ADV
# 7    一号企业  ni   8  ATT
# 8      先生   n   6  POB
# 9      签署   v   0  HED
# 10      了   u   9  RAD
# print(parse('2016年 ner_1614_ 有限公司 瑞峰 张家港 光伏 科技 有限公司 支付 设备 款 人民币 4 515 770.00 元'))
# print(parse('2016年 ner_1614_ 有限公司瑞峰张家港光伏科技有限公司支付设备款人民币4515770.00元'))

# print(parse('资产 出售 资产 购买 方案 简要 介绍 资产 出售 公司 拟 控股 股东 ner_1625_ 出售 ner_1463_ 100% 股权'))
# print(parse('资产出售资产购买方案简要介绍资产出售公司拟控股股东 ner_1625_ 出售 ner_1463_ 100%股权'))

# print(parse('集团 ner_1089_ 股份 有限公司 持有 ner_1616_ 股份 有限公司 股票 本期 变动 系 买卖 一揽子 沪 深 300 指数 成份股 致'))
# print(parse('本集团及 ner_1089_ 股份有限公司持有 ner_1616_ 大股份有限公司股票的本期变动系买卖一揽子沪深300指数成份股所致'))


# TODO：汇总上述函数汇总句法分析特征与TFIDF特征
# TODO ...
def get_extra_feature(corpus):
    """
    获取语料数据的额外特征
    """
    # 初始化存放所有语料数据额外特征的列表
    extra_features_list = []
    # 遍历语料，计算各条数据对应的额外特征，并添加到列表
    try:
        with tqdm(corpus) as t:
            for i in t:
                current_extra_features = parse(i)
                extra_features_list.append(current_extra_features)
        return np.array(extra_features_list)
    except KeyboardInterrupt:
        t.close()
        raise


# 对训练解和测试集分别拼接tf-idf特征与额外特征
# # 这里直接用corpus(主要是为了去除了停用词，当然为了提取tf-idf弄成了空格分词形式)的数据进行提取额外特征
# # (注：空格分词并会不影响提取的额外特征)
X_train_extra_features = get_extra_feature(corpus_train)
X_test_extra_features = get_extra_feature(corpus_test)

X_train = sparse.hstack((X_train, sparse.csr_matrix(X_train_extra_features)), format='csr')
X_test = sparse.hstack((X_test, sparse.csr_matrix(X_test_extra_features)), format='csr')

print(X_train.shape)
print(X_test.shape)

100%|██████████| 419/419 [00:56<00:00,  7.63it/s]

(850, 4577)
(419, 4577)


### 练习5：建立分类器

利用已经提取好的tfidf特征以及parse特征，建立分类器进行分类任务。

In [23]:
# 建立分类器进行分类
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# TODO：定义需要遍历的参数
tuned_parameters ={"C": [0.001, 0.003, 0.01, 0.003, 0.1, 0.3, 1, 3]}
scores = ['precision', 'recall', "f1"]

# TODO：选择模型
lr = LogisticRegression()

# TODO：利用GridSearchCV搜索最佳参数
for score in scores:
    print("# Tuning hyper-parameters for %s---------" % score)
    print()

    clf = GridSearchCV(LogisticRegression(),
                       tuned_parameters,
                       cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_pred_train = clf.predict(X_train)
    print(classification_report(y_train, y_pred_train))
    print()

# TODO：对Test_data进行分类
best_model = clf.best_estimator_
y_pred_test = best_model.predict(X_test)



# Tuning hyper-parameters for precision---------



/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are u

Best parameters set found on development set:

{'C': 3}

Grid scores on development set:

0.000 (+/-0.000) for {'C': 0.001}
0.000 (+/-0.000) for {'C': 0.003}
0.000 (+/-0.000) for {'C': 0.01}
0.000 (+/-0.000) for {'C': 0.003}
0.666 (+/-0.844) for {'C': 0.1}
0.950 (+/-0.200) for {'C': 0.3}
0.973 (+/-0.106) for {'C': 1}
0.980 (+/-0.080) for {'C': 3}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       722
           1       1.00      0.95      0.97       128

    accuracy                           0.99       850
   macro avg       1.00      0.97      0.98       850
weighted avg       0.99      0.99      0.99       850


# Tuning hyper-parameters for recall---------



/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'pre

Best parameters set found on development set:

{'C': 3}

Grid scores on development set:

0.000 (+/-0.000) for {'C': 0.001}
0.000 (+/-0.000) for {'C': 0.003}
0.000 (+/-0.000) for {'C': 0.01}
0.000 (+/-0.000) for {'C': 0.003}
0.039 (+/-0.051) for {'C': 0.1}
0.204 (+/-0.110) for {'C': 0.3}
0.524 (+/-0.156) for {'C': 1}
0.719 (+/-0.194) for {'C': 3}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       722
           1       1.00      0.95      0.97       128

    accuracy                           0.99       850
   macro avg       1.00      0.97      0.98       850
weighted avg       0.99      0.99      0.99       850


# Tuning hyper-parameters for f1---------

Best parameters set found on development set:

{'C': 3}

Grid scores on development set:

0.000 (+/-0.000) for {'C': 0.001}
0.000 (+/-0.000) fo

/home/zing/vir/tf_py3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [25]:
# TODO：保存结果
# 答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
# info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
# info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。

# 实体编号对应表格
# # 初始化结果列表
info_extract_entity_list = []

# # 遍历ner_dict_new, ner_dict_reverse_new进行生成
for k in ner_dict_new:
    entity_id = ner_dict_new[k]
    entity_names = '|'.join(ner_dict_reverse_new[k])
    info_extract_entity_list.append([entity_id, entity_names])

# # 生成表格
info_extract_entity_df = pd.DataFrame(info_extract_entity_list, columns=['实体编号', '实体名'])
info_extract_entity_df.to_csv('../submit/info_extract_entity.csv', index=False)

# 实体联系对应表格
# # 初始化最终的结果列表
info_extract_submit_list = []
# # 先利用预测的标签，提取出相应的句子
predict_positive_examples = pd.Series(corpus_test)[pd.Series(y_pred_test).astype(bool)]
# # 遍历各个句子
for example in predict_positive_examples:
    # # 正则提取该句子所有的实体符号，并去重
    current_entity_list = re.findall('ner_(\d\d\d\d)_', example)
    current_entity_set = set(current_entity_list)
    # 当前句子的实体组合关系
    current_entity_pair_list = list(combinations(current_entity_set, 2))
    # 将当前句子的实体组合关系扩充到结果列表中
    info_extract_submit_list.extend(current_entity_pair_list)

# # 生成表格
info_extract_submit_df = pd.DataFrame(info_extract_submit_list, columns=['实体1', '实体2'])
info_extract_submit_df.to_csv('../submit/info_extract_submit.csv', index=False)

In [28]:
current_entity_pair_list

[('1608', '1609')]

### 练习6：操作图数据库
对关系最好的描述就是用图，那这里就需要使用图数据库，目前最常用的图数据库是noe4j，通过cypher语句就可以操作图数据库的增删改查。可以参考“https://cuiqingcai.com/4778.html”。

本次作业我们使用neo4j作为图数据库，neo4j需要java环境，请先配置好环境。

将我们提出的实体关系插入图数据库，并查询某节点的3层投资关系。

In [ ]:

from py2neo import Node, Relationship, Graph

graph = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="person"
)

for v in relation_list:
    a = Node('Company', name=v[0])
    b = Node('Company', name=v[1])
    
    # 本次不区分投资方和被投资方，无向图
    r = Relationship(a, 'INVEST', b)
    s = a | b | r
    graph.create(s)
    r = Relationship(b, 'INVEST', a)
    s = a | b | r
    graph.create(s)

In [ ]:
# TODO：查询某节点的3层投资关系
# TODO：查询某节点的3层投资关系
graph.run("match data=(na:Company{name:'1018'})-[*1..3]->(nb:Company) return data").data()

## 步骤4：实体消歧
解决了实体识别和关系的提取，我们已经完成了一大截，但是我们提取的实体究竟对应知识库中哪个实体呢？下图中，光是“苹果”就对应了13个同名实体。
<img src="../image/baike2.png", width=340, heigth=480>

在这个问题上，实体消歧旨在解决文本中广泛存在的名称歧义问题，将句中识别的实体与知识库中实体进行匹配，解决实体歧义问题。


### 练习7：
匹配test_data.csv中前25条样本中的人物实体对应的百度百科URL（此部分样本中所有人名均可在百度百科中链接到）。

利用scrapy、beautifulsoup、request等python包对百度百科进行爬虫，判断是否具有一词多义的情况，如果有的话，选择最佳实体进行匹配。

使用URL为‘https://baike.baidu.com/item/’+人名 可以访问百度百科该人名的词条，此处需要根据爬取到的网页识别该词条是否对应多个实体，如下图：
<img src="../image/baike1.png", width=440, heigth=480>
如果该词条有对应多个实体，请返回正确匹配的实体URL，例如该示例网页中的‘https://baike.baidu.com/item/陆永/20793929’。

- 提交文件：entity_disambiguation_submit.csv
- 提交格式：第一列为实体id（与info_extract_submit.csv中id保持一致），第二列为对应URL。
- 示例：

| 实体编号 | URL |
| ------ | ------ |
| 1001 | https://baike.baidu.com/item/陆永/20793929 |
| 1002 | https://baike.baidu.com/item/王芳/567232 |


In [33]:
import jieba
import pandas as pd
import requests
from bs4 import BeautifulSoup

from copy import copy
import fool
import numpy as np


# 找出test_data.csv中前25条样本所有的人物名称，以及人物所在文档的上下文内容
test_data = pd.read_csv('../data/info_extract/test_data.csv', encoding = 'gb2312', header=0)

# 存储人物以及上下文信息（key为人物ID，value为人物名称、人物上下文内容）
person_name = {}

# 观察上下文的窗口大小
window = 10  

# 遍历前25条样本
for i in range(25):
    sentence = copy(test_data.iloc[i, 1])
    words, ners = fool.analysis(sentence)
    ners[0].sort(key=lambda x:x[0], reverse=True)
    for start, end, ner_type, ner_name in ners[0]:
        if ner_type=='person':
            # TODO：提取实体的上下文
            context = sentence[max(0, start-window):start] + sentence[end-1:min(len(sentence), end-1+window)]
            person_name[i] = (ner_name, context)

# print(person_name)

def calculate_sentence_similarity(string1, string2):
    """
    计算两个句子的相似度(基于unigram, bigram, trigram的重复程度)
    """
    # unigram
    unigram_set1 = set(string1)
    unigram_set2 = set(string2)
    unigram_intersection = unigram_set1 & unigram_set2
    unigram_union = unigram_set1 | unigram_set2
    unigram_similarity_score = len(unigram_intersection) / len(unigram_union)
    
    # bigram
    bigram_set1 = {string1[idx:idx+2] for idx in range(len(string1)-1)}
    bigram_set2 = {string2[idx:idx+2] for idx in range(len(string2)-1)}
    bigram_intersection = bigram_set1 & bigram_set2
    bigram_union = bigram_set1 | bigram_set2
    bigram_similarity_score = len(bigram_intersection) / len(bigram_union)
    
    # trigram
    trigram_set1 = {string1[idx:idx+3] for idx in range(len(string1)-2)}
    trigram_set2 = {string2[idx:idx+3] for idx in range(len(string2)-2)}
    trigram_intersection = trigram_set1 & trigram_set2
    trigram_union = trigram_set1 | trigram_set2
    trigram_similarity_score = len(trigram_intersection) / len(trigram_union)
    
    return unigram_similarity_score + bigram_similarity_score + trigram_similarity_score

# 利用爬虫得到每个人物名称对应的URL
# TODO：找到每个人物实体的词条内容。
# # 初始化结果列表
result_data = []
# # 循环爬取

try:
    with tqdm(person_name.items()) as tq:
        for k,v in tq:
            # 拼接生成serach_url
            current_name = v[0]
            search_url = 'https://baike.baidu.com/item/' + current_name
            # 获取当前人名对应实体编号
            current_name_id = ner_dict_new[current_name]
        #     print(current_name_id, current_name)
            # 请求数据
            headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
            web_data = requests.get(search_url, headers=headers)
            web_data.encoding = 'utf-8'
            # 解析数据
            soup = BeautifulSoup(web_data.text, 'html.parser')
            # # 查找是否有百科搜到的title
            check_title = soup.select('dd.lemmaWgt-lemmaTitle-title > h1')

            if check_title:
                # TODO：将样本中人物上下文与爬取词条结果进行对比，选择最接近的词条
                current_context = v[1]
                # 获取对应节点列表
                entries_node = soup.select('li.item *')
                # 获取词条列表
                entries_list = [i.text for i in entries_node]
        #         print(entries_list)
                # 获取词条链接列表
                entries_url_list = [search_url if i.get('class') else 'https://baike.baidu.com'+i.get('href') for i in entries_node]
                # 判断是否获取到了词条
                if entries_list:
                    # # 计算所有词条与当前人物名称的相似度
                    entries_similarity_list = [calculate_sentence_similarity(current_context, entry) for entry in entries_list]
#                     print(entries_similarity_list)
                    # # 获取最大相似度所在的索引
                    max_similarity_index = np.argmax(entries_similarity_list)
                    # # 获取最大相似度对应的词条链接
                    max_similarity_entry_url = entries_url_list[max_similarity_index]
                    # # 将该词条链接添加到结果列表
                    result_data.append((current_name_id, max_similarity_entry_url))
                else:
                    result_data.append((current_name_id, search_url))
            else:
                result_data.append((current_name_id, 'N/A'))
except KeyboardInterrupt:
    tq.close()
    raise
                
# print(result_data)

# 输出结果
result_df = pd.DataFrame(result_data, columns=['实体编号', 'URL'])
result_df.to_csv('../submit/entity_disambiguation_submit.csv', index=False)
result_df

100%|██████████| 25/25 [00:13<00:00,  2.04it/s]


,实体编号,URL
0,1001,https://baike.baidu.com/item/%E6%9D%8E%E4%BA%9...
1,1003,https://baike.baidu.com/item/%E4%BE%AF%E6%AF%8...
2,1005,https://baike.baidu.com/item/%E5%BC%A0%E5%BE%B...
3,1007,https://baike.baidu.com/item/%E8%82%96%E6%96%8...
4,1009,https://baike.baidu.com/item/%E7%86%8A%E6%B5%B...
5,1011,https://baike.baidu.com/item/%E5%AE%8B%E7%90%B...
6,1014,https://baike.baidu.com/item/王友林
7,1016,https://baike.baidu.com/item/彭聪
8,1017,https://baike.baidu.com/item/%E6%9B%B9%E9%A3%9...
9,1019,https://baike.baidu.com/item/%E9%A2%9C%E5%86%9...
